In [26]:
import tkinter as tk
from tkinter import ttk, messagebox
import math

# -----------------------------
# Theme Colors
# -----------------------------
BG_COLOR = "#121212"       # Dark background
BTN_COLOR = "#1E1E1E"      # Button background
BTN_HOVER = "#2C2C2C"      # Hover color
FG_COLOR = "#00FFC6"       # Neon cyan text
ACCENT_COLOR = "#FF3C98"   # Neon pink

FONT_MAIN = ("Segoe UI", 16)
FONT_BTN = ("Segoe UI", 14, "bold")
FONT_SMALL = ("Segoe UI", 12)

# -----------------------------
# Calculator Class
# -----------------------------
class FuturisticCalculator(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Scientific Calculator")
        self.geometry("500x650")
        self.config(bg=BG_COLOR)
        self.resizable(True, True)

        self.memory = 0

        # converter sliding state and width
        self.converter_visible = False
        self.converter_width = 360  # width of sliding panel in px (adjustable)
        self._anim_after_id = None

        self.create_display()
        self.create_buttons()
        self.create_converter_panel()

        # Keep panel off-screen if window resized
        self.bind("<Configure>", self._on_window_configure)

    def create_display(self):
        self.display_var = tk.StringVar()
        self.display = tk.Entry(self, textvariable=self.display_var, font=("Consolas", 22),
                                bg=BG_COLOR, fg=FG_COLOR, bd=0, justify="right", insertbackground=FG_COLOR)
        self.display.pack(fill="x", padx=10, pady=10, ipady=15)

    def create_buttons(self):
        btn_frame = tk.Frame(self, bg=BG_COLOR)
        btn_frame.pack(expand=True, fill="both")

        buttons = [
            ["MC", "MR", "M+", "M-", "C"],
            ["sin", "cos", "tan", "log", "√"],
            ["7", "8", "9", "/", "^"],
            ["4", "5", "6", "*", "("],
            ["1", "2", "3", "-", ")"],
            ["0", ".", "π", "+", "="],
        ]

        for r, row in enumerate(buttons):
            btn_row = tk.Frame(btn_frame, bg=BG_COLOR)
            btn_row.pack(fill="x", expand=True)
            for c, char in enumerate(row):
                b = tk.Button(btn_row, text=char, font=FONT_BTN, fg=FG_COLOR, bg=BTN_COLOR,
                              activebackground=BTN_HOVER, activeforeground=ACCENT_COLOR,
                              bd=0, relief="flat", command=lambda ch=char: self.on_button_click(ch))
                b.pack(side="left", expand=True, fill="both", padx=2, pady=2)

        # Converter button stays in the same place as before
        conv_btn = tk.Button(self, text="Converter", font=FONT_BTN, fg=ACCENT_COLOR,
                             bg=BTN_COLOR, bd=0, relief="flat",
                             command=self.toggle_converter)
        conv_btn.pack(fill="x", padx=10, pady=(0, 10))

    def create_converter_panel(self):
        """Create the converter UI inside a frame that will be placed off-screen initially.
           The internal UI is unchanged from your version."""
        # frame configured for sliding (we'll use place on it)
        self.converter_frame = tk.Frame(self, bg="#1A1A1A", width=self.converter_width)
        # build the inner UI exactly as before
        self.unit_categories = {
            "Length": ["Meter", "Kilometer", "Centimeter", "Millimeter", "Mile", "Yard", "Foot", "Inch"],
            "Weight": ["Kilogram", "Gram", "Milligram", "Pound", "Ounce"],
            "Temperature": ["Celsius", "Fahrenheit", "Kelvin"],
            "Speed": ["m/s", "km/h", "mph", "knot"],
            "Area": ["Square meter", "Square kilometer", "Square mile", "Square yard", "Square foot", "Acre", "Hectare"],
            "Volume": ["Liter", "Milliliter", "Cubic meter", "Cubic centimeter", "Gallon", "Pint"],
            "Time": ["Second", "Minute", "Hour", "Day"],
            "Energy": ["Joule", "Kilojoule", "Calorie", "Kilocalorie", "Watt-hour", "Kilowatt-hour"],
            "Pressure": ["Pascal", "Kilopascal", "Bar", "PSI", "Atmosphere"],
            "Data": ["Bit", "Byte", "Kilobyte", "Megabyte", "Gigabyte", "Terabyte"]
        }

        self.cat_var = tk.StringVar(value="Length")
        self.from_unit = tk.StringVar()
        self.to_unit = tk.StringVar()
        self.input_value = tk.StringVar()
        self.output_value = tk.StringVar()

        # Dropdown for category
        cat_menu = ttk.Combobox(self.converter_frame, values=list(self.unit_categories.keys()),
                                textvariable=self.cat_var, state="readonly")
        cat_menu.pack(pady=8, padx=10, fill="x")
        cat_menu.bind("<<ComboboxSelected>>", self.update_units)

        # Unit dropdowns
        self.from_menu = ttk.Combobox(self.converter_frame, values=self.unit_categories["Length"],
                                      textvariable=self.from_unit, state="readonly")
        self.from_menu.pack(pady=6, padx=10, fill="x")
        self.to_menu = ttk.Combobox(self.converter_frame, values=self.unit_categories["Length"],
                                    textvariable=self.to_unit, state="readonly")
        self.to_menu.pack(pady=6, padx=10, fill="x")

        # Input
        tk.Entry(self.converter_frame, textvariable=self.input_value, font=FONT_MAIN,
                 bg="#101010", fg=FG_COLOR, bd=0, insertbackground=FG_COLOR).pack(pady=8, padx=10, fill="x")
        tk.Button(self.converter_frame, text="Convert", font=FONT_BTN, fg=FG_COLOR, bg=BTN_COLOR,
                  command=self.convert_units).pack(pady=8, padx=10, fill="x")

        # Output
        tk.Entry(self.converter_frame, textvariable=self.output_value, font=FONT_MAIN,
                 state="readonly", bg="#101010", fg=FG_COLOR).pack(pady=6, padx=10, fill="x")

        # ensure default units populated
        self.update_units()

        # place it initially off-screen to the right
        self.update_idletasks()
        win_w = self.winfo_width()
        win_h = self.winfo_height()
        # If window not yet fully initialized, schedule an initial placement shortly
        if win_w <= 1:
            self.after(50, self._place_panel_offscreen)
        else:
            self._place_panel_offscreen()

    def _place_panel_offscreen(self):
        """Place converter off-screen to the right (initial state)."""
        win_w = self.winfo_width()
        win_h = self.winfo_height()
        # place panel to cover full height on right when visible, but off-screen now
        self.converter_frame.place(x=win_w, y=0, width=self.converter_width, height=win_h)

    def _on_window_configure(self, event=None):
        """Keep converter placement consistent while resizing (if it's hidden, keep off-screen)."""
        # if converter visible, ensure it's placed at right x (window_width - converter_width)
        if self.converter_visible:
            win_w = self.winfo_width()
            win_h = self.winfo_height()
            target_x = max(win_w - self.converter_width, 0)
            self.converter_frame.place(x=target_x, y=0, width=self.converter_width, height=win_h)
        else:
            # keep off-screen
            win_w = self.winfo_width()
            win_h = self.winfo_height()
            self.converter_frame.place(x=win_w, y=0, width=self.converter_width, height=win_h)

    def toggle_converter(self):
        """Toggle panel with slide animation (fast & snappy)."""
        if self._anim_after_id:
            # prevent double toggles during animation
            return
        if self.converter_visible:
            self._slide_out()
        else:
            self._slide_in()

    def _slide_in(self):
        """Animate sliding in from right to (win_w - converter_width)."""
        self.converter_visible = True
        self.converter_frame.lift()  # ensure on top
        self.update_idletasks()
        win_w = self.winfo_width()
        win_h = self.winfo_height()
        start_x = win_w
        target_x = max(win_w - self.converter_width, 0)
        step = max(20, (start_x - target_x) // 12)  # ensure fast & snappy (~200-300ms)
        # place at start
        self.converter_frame.place(x=start_x, y=0, width=self.converter_width, height=win_h)

        def anim(x):
            if x <= target_x:
                self.converter_frame.place(x=target_x, y=0, width=self.converter_width, height=win_h)
                self._anim_after_id = None
                return
            x_next = x - step
            self.converter_frame.place(x=x_next, y=0, width=self.converter_width, height=win_h)
            self._anim_after_id = self.after(12, lambda: anim(x_next))

        anim(start_x)

    def _slide_out(self):
        """Animate sliding out to the right (off-screen)."""
        self.update_idletasks()
        win_w = self.winfo_width()
        win_h = self.winfo_height()
        start_x = max(win_w - self.converter_width, 0)
        end_x = win_w
        step = max(20, (end_x - start_x) // 12)
        def anim(x):
            if x >= end_x:
                # final cleanup: place off-screen
                self.converter_frame.place(x=end_x, y=0, width=self.converter_width, height=win_h)
                self.converter_visible = False
                self._anim_after_id = None
                return
            x_next = x + step
            self.converter_frame.place(x=x_next, y=0, width=self.converter_width, height=win_h)
            self._anim_after_id = self.after(12, lambda: anim(x_next))
        anim(start_x)

    def update_units(self, event=None):
        cat = self.cat_var.get()
        units = self.unit_categories[cat]
        self.from_menu["values"] = units
        self.to_menu["values"] = units
        # set sensible defaults if empty
        if units:
            # only change selection if not already set
            if not self.from_unit.get():
                self.from_unit.set(units[0])
            if not self.to_unit.get() or self.to_unit.get() not in units:
                # choose second if possible
                self.to_unit.set(units[1] if len(units) > 1 else units[0])

    def convert_units(self):
        try:
            val = float(self.input_value.get())
            from_u = self.from_unit.get()
            to_u = self.to_unit.get()
            cat = self.cat_var.get()
            result = self.unit_convert_logic(cat, from_u, to_u, val)
            self.output_value.set(str(result))
        except ValueError:
            messagebox.showerror("Error", "Invalid number.")

    def unit_convert_logic(self, category, from_u, to_u, value):
        if category == "Length":
            factors = {"Meter": 1, "Kilometer": 1000, "Centimeter": 0.01, "Millimeter": 0.001,
                       "Mile": 1609.34, "Yard": 0.9144, "Foot": 0.3048, "Inch": 0.0254}
            return value * factors[from_u] / factors[to_u]
        if category == "Weight":
            factors = {"Kilogram": 1, "Gram": 0.001, "Milligram": 1e-6,
                       "Pound": 0.453592, "Ounce": 0.0283495}
            return value * factors[from_u] / factors[to_u]
        if category == "Temperature":
            if from_u == "Celsius":
                if to_u == "Fahrenheit": return value * 9/5 + 32
                if to_u == "Kelvin": return value + 273.15
            if from_u == "Fahrenheit":
                if to_u == "Celsius": return (value - 32) * 5/9
                if to_u == "Kelvin": return (value - 32) * 5/9 + 273.15
            if from_u == "Kelvin":
                if to_u == "Celsius": return value - 273.15
                if to_u == "Fahrenheit": return (value - 273.15) * 9/5 + 32
            return value
        if category == "Speed":
            factors = {"m/s": 1, "km/h": 1000/3600, "mph": 1609.34/3600, "knot": 1852/3600}
            return value * factors[from_u] / factors[to_u]
        if category == "Area":
            factors = {"Square meter": 1, "Square kilometer": 1e6, "Square mile": 2.59e6,
                       "Square yard": 0.836127, "Square foot": 0.092903, "Acre": 4046.86, "Hectare": 1e4}
            return value * factors[from_u] / factors[to_u]
        if category == "Volume":
            factors = {"Liter": 1, "Milliliter": 0.001, "Cubic meter": 1000, "Cubic centimeter": 0.001,
                       "Gallon": 3.78541, "Pint": 0.473176}
            return value * factors[from_u] / factors[to_u]
        if category == "Time":
            factors = {"Second": 1, "Minute": 60, "Hour": 3600, "Day": 86400}
            return value * factors[from_u] / factors[to_u]
        if category == "Energy":
            factors = {"Joule": 1, "Kilojoule": 1000, "Calorie": 4.184, "Kilocalorie": 4184,
                       "Watt-hour": 3600, "Kilowatt-hour": 3.6e6}
            return value * factors[from_u] / factors[to_u]
        if category == "Pressure":
            factors = {"Pascal": 1, "Kilopascal": 1000, "Bar": 1e5, "PSI": 6894.76, "Atmosphere": 101325}
            return value * factors[from_u] / factors[to_u]
        if category == "Data":
            factors = {"Bit": 1, "Byte": 8, "Kilobyte": 8*1024, "Megabyte": 8*1024**2,
                       "Gigabyte": 8*1024**3, "Terabyte": 8*1024**4}
            return value * factors[from_u] / factors[to_u]
        return value

    def on_button_click(self, char):
        if char == "C":
            self.display_var.set("")
        elif char == "=":
            try:
                expression = self.display_var.get()
                expression = expression.replace("π", str(math.pi)).replace("^", "**")
                expression = expression.replace("√", "math.sqrt")
                result = eval(expression, {"math": math, "__builtins__": None})
                self.display_var.set(str(result))
            except Exception:
                self.display_var.set("Error")
        elif char in ["sin", "cos", "tan", "log"]:
            self.display_var.set(self.display_var.get() + f"math.{char}(")
        elif char in ["MC", "MR", "M+", "M-"]:
            self.memory_operations(char)
        else:
            self.display_var.set(self.display_var.get() + char)

    def memory_operations(self, op):
        try:
            current = float(self.display_var.get())
            if op == "MC": self.memory = 0
            elif op == "MR": self.display_var.set(str(self.memory))
            elif op == "M+": self.memory += current
            elif op == "M-": self.memory -= current
        except ValueError:
            pass

if __name__ == "__main__":
    app = FuturisticCalculator()
    app.mainloop()

In [ ]:
import tkinter as tk
from tkinter import ttk
import math

# ---------------------------
# UNIT DATA
# ---------------------------
unit_categories = {
    "Length": {
        "m": 1,
        "km": 1000,
        "cm": 0.01,
        "mm": 0.001,
        "mile": 1609.34,
        "yard": 0.9144,
        "foot": 0.3048,
        "inch": 0.0254
    },
    "Mass": {
        "kg": 1,
        "g": 0.001,
        "mg": 1e-6,
        "ton": 1000,
        "lb": 0.453592,
        "oz": 0.0283495
    },
    "Temperature": {},  # handled separately
    "Time": {
        "second": 1,
        "minute": 60,
        "hour": 3600,
        "day": 86400
    },
    "Area": {
        "m²": 1,
        "km²": 1e6,
        "cm²": 0.0001,
        "mm²": 1e-6,
        "acre": 4046.86,
        "hectare": 10000
    },
    "Volume": {
        "m³": 1,
        "liter": 0.001,
        "ml": 1e-6,
        "gallon": 0.00378541,
        "cup": 0.000236588
    },
    "Speed": {
        "m/s": 1,
        "km/h": 1000/3600,
        "mph": 1609.34/3600,
        "knot": 1852/3600
    },
    "Pressure": {
        "Pa": 1,
        "kPa": 1000,
        "bar": 100000,
        "psi": 6894.76
    },
    "Energy": {
        "J": 1,
        "kJ": 1000,
        "cal": 4.184,
        "kcal": 4184,
        "Wh": 3600,
        "kWh": 3.6e6
    },
    "Power": {
        "W": 1,
        "kW": 1000,
        "MW": 1e6,
        "hp": 745.7
    },
    "Frequency": {
        "Hz": 1,
        "kHz": 1000,
        "MHz": 1e6,
        "GHz": 1e9
    },
    "Data Storage": {
        "bit": 1,
        "byte": 8,
        "KB": 8 * 1024,
        "MB": 8 * 1024**2,
        "GB": 8 * 1024**3,
        "TB": 8 * 1024**4
    },
    "Fuel Economy": {
        "mpg": 1,
        "km/l": 2.35215,
        "l/100km": None  # needs inverse handling
    },
    "Angle": {
        "degree": 1,
        "radian": 180 / math.pi,
        "grad": 0.9
    }
}

# ---------------------------
# CONVERSION FUNCTION
# ---------------------------
def convert_units(value, from_unit, to_unit, category):
    try:
        val = float(value)
    except:
        return ""

    if category == "Temperature":
        if from_unit == "°C":
            if to_unit == "°F": return f"{val * 9/5 + 32:.6g}"
            elif to_unit == "K": return f"{val + 273.15:.6g}"
            else: return f"{val:.6g}"
        elif from_unit == "°F":
            c = (val - 32) * 5/9
            return convert_units(c, "°C", to_unit, category)
        elif from_unit == "K":
            c = val - 273.15
            return convert_units(c, "°C", to_unit, category)
    else:
        factor_dict = unit_categories[category]
        from_factor = factor_dict[from_unit]
        to_factor = factor_dict[to_unit]
        if category == "Fuel Economy" and (from_unit == "l/100km" or to_unit == "l/100km"):
            if from_unit == "l/100km":
                km_per_l = 100 / val
                return convert_units(km_per_l, "km/l", to_unit, category)
            elif to_unit == "l/100km":
                km_per_l = val / factor_dict[from_unit]
                return f"{100 / km_per_l:.6g}"
        else:
            meters = val * from_factor
            converted = meters / to_factor
            return f"{converted:.6g}"
    return ""

# ---------------------------
# MAIN APP
# ---------------------------
root = tk.Tk()
root.title("Futuristic Calculator with Unit Converter")
root.geometry("420x600")
root.config(bg="#1E1E2F")

# ---------------------------
# CALCULATOR FRAME
# ---------------------------
calc_frame = tk.Frame(root, bg="#1E1E2F")
calc_frame.pack(fill="both", expand=True)

expr = ""
display_var = tk.StringVar()

def press(num):
    global expr
    expr += str(num)
    display_var.set(expr)

def equalpress():
    global expr
    try:
        total = str(eval(expr))
        display_var.set(total)
        expr = total
    except:
        display_var.set("error")
        expr = ""

def clear():
    global expr
    expr = ""
    display_var.set("")

display = tk.Entry(calc_frame, font=("Segoe UI", 20), textvariable=display_var, bd=0, bg="#2D2D44", fg="white", justify="right")
display.pack(fill="x", ipady=10, pady=10, padx=10)

btns = [
    ("7", lambda: press(7)), ("8", lambda: press(8)), ("9", lambda: press(9)), ("/", lambda: press("/")),
    ("4", lambda: press(4)), ("5", lambda: press(5)), ("6", lambda: press(6)), ("*", lambda: press("*")),
    ("1", lambda: press(1)), ("2", lambda: press(2)), ("3", lambda: press(3)), ("-", lambda: press("-")),
    ("0", lambda: press(0)), (".", lambda: press(".")), ("C", clear), ("+", lambda: press("+")),
    ("=", equalpress)
]

btn_frame = tk.Frame(calc_frame, bg="#1E1E2F")
btn_frame.pack()

for i, (text, cmd) in enumerate(btns):
    tk.Button(btn_frame, text=text, command=cmd, font=("Segoe UI", 14), width=5, height=2,
              bg="#3C3C5C", fg="white", bd=0).grid(row=i//4, column=i%4, padx=5, pady=5)

# ---------------------------
# CONVERTER PANEL
# ---------------------------
converter_width = 300
converter_frame = tk.Frame(root, bg="#26263A", width=converter_width, height=600)
converter_frame.place(x=420, y=0)  # start hidden

def update_units(*args):
    category = category_var.get()
    unit_list = list(unit_categories[category].keys()) if category != "Temperature" else ["°C", "°F", "K"]
    from_unit_cb["values"] = unit_list
    to_unit_cb["values"] = unit_list
    from_unit_cb.current(0)
    to_unit_cb.current(1)
    update_conversion()

def update_conversion(*args):
    result = convert_units(from_val.get(), from_unit_var.get(), to_unit_var.get(), category_var.get())
    to_val.set(result)

# Category selection
category_var = tk.StringVar()
category_cb = ttk.Combobox(converter_frame, textvariable=category_var, state="readonly", font=("Segoe UI", 12))
category_cb["values"] = list(unit_categories.keys())
category_cb.current(0)
category_cb.pack(pady=10, padx=10, fill="x")
category_var.trace("w", update_units)

# From value + unit
from_frame = tk.Frame(converter_frame, bg="#26263A")
from_frame.pack(pady=5, padx=10, fill="x")
from_val = tk.StringVar()
from_val.trace("w", update_conversion)
from_entry = tk.Entry(from_frame, textvariable=from_val, font=("Segoe UI", 12), bg="#3C3C5C", fg="white", relief="flat")
from_entry.pack(side="left", fill="x", expand=True)
from_unit_var = tk.StringVar()
from_unit_cb = ttk.Combobox(from_frame, textvariable=from_unit_var, state="readonly", font=("Segoe UI", 12))
from_unit_cb.pack(side="right")
from_unit_var.trace("w", update_conversion)

# To value + unit
to_frame = tk.Frame(converter_frame, bg="#26263A")
to_frame.pack(pady=5, padx=10, fill="x")
to_val = tk.StringVar()
to_entry = tk.Entry(to_frame, textvariable=to_val, font=("Segoe UI", 12), bg="#3C3C5C", fg="white", relief="flat", state="readonly")
to_entry.pack(side="left", fill="x", expand=True)
to_unit_var = tk.StringVar()
to_unit_cb = ttk.Combobox(to_frame, textvariable=to_unit_var, state="readonly", font=("Segoe UI", 12))
to_unit_cb.pack(side="right")
to_unit_var.trace("w", update_conversion)

# ---------------------------
# SLIDE ANIMATION
# ---------------------------
is_open = False
def toggle_converter():
    global is_open
    if is_open:
        for x in range(420 - converter_width, 421, 20):
            converter_frame.place(x=420 + (420 - x), y=0)
            converter_frame.update()
        is_open = False
    else:
        for x in range(420, 420 - converter_width - 1, -20):
            converter_frame.place(x=x, y=0)
            converter_frame.update()
        is_open = True

toggle_btn = tk.Button(calc_frame, text="Converter", command=toggle_converter, font=("Segoe UI", 12),
                       bg="#5A5A7A", fg="white", relief="flat")
toggle_btn.pack(pady=5)

update_units()

root.mainloop()